In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import warnings
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

data = pd.read_csv('fifth-run/feats_version_5_22_acc.csv')
data = data[~data["activity"].str.contains("jogging")]
data["activity"] = data["activity"].replace(["sitting", "lying"], "resting")
result = data[data["name"].str.contains("kuba")]
result_train = data[data["name"].str.contains("kuba") == False]
result_train.drop('name', axis=1, inplace=True)
result.drop('name', axis=1, inplace=True)
X_train = result_train.iloc[:, :-1].values
y_train = result_train.iloc[:, -1].values
X_test = result.iloc[:, :-1].values
y_test = result.iloc[:, -1].values
X_val, X_test, y_val, y_test= train_test_split(X_test, y_test, test_size=0.5, random_state=42)

label_encoder = LabelEncoder()
labels_train1 = label_encoder.fit_transform(y_train)
labels_test1 = label_encoder.transform(y_test)

In [ ]:
classifier = joblib.load('XGBoost.h5')
y_pred = classifier.predict(X_test)

class_mapping = {
    0: "schodzenie",
    1: "odpoczynek",
    2: "przysiady",
    3: "stanie",
    4: "wchodzenie",
    5:  "chodzenie"  }

listt =[]
for key in class_mapping.values():
    listt.append(key)
    
print(f"Accuracy:",accuracy_score(labels_test1, y_pred))
print("Overall Classification Report:\n", classification_report(labels_test1, y_pred,target_names=listt))
confusion_mtx = confusion_matrix(labels_test1, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mtx, annot=True, fmt='d', cmap='Blues', xticklabels=listt, yticklabels=listt)
plt.xlabel('Przewidziana aktywność')
plt.ylabel('Rzeczywista aktywność')
plt.title('Macierz błędów')
plt.show()